# smart attendance system. Here's a simplified breakdown:

1. **Loading Images**: It starts by loading images of known faces from a specified directory. These images are used as reference points for recognizing faces later on.

2. **Recognizing Faces**: It then loads a group photo and identifies faces within it. For each face detected, it compares it with the known faces using a technique called face recognition.

3. **Attendance Marking**: If a known face is recognized in the group photo, it marks the person as present in the attendance record along with the current time.

4. **Saving Attendance**: Finally, it saves the attendance records to an Excel file with timestamps.

So basically, it's a system that checks who's present in a group photo based on pre-registered faces and records their attendance along with the time.

Sure, here's a breakdown of the main libraries and model used in the code:

1. **OpenCV (cv2)**: OpenCV is a popular library for computer vision tasks. In this code, it's used for loading and manipulating images, including resizing and extracting faces from the group photo.

2. **NumPy (np)**: NumPy is a fundamental library for numerical computing in Python. It's used here for various array operations, especially in handling face encodings and distances.

3. **Pandas (pd)**: Pandas is a powerful library for data manipulation and analysis. It's used to create and manage the attendance DataFrame, which stores information about the attendance status of individuals.

4. **Face Recognition**: This code utilizes the face recognition library, which is built on top of dlib and provides face recognition capabilities. It's used to recognize faces in both the known images and the group photo, comparing face encodings to determine matches.

5. **datetime**: The datetime module provides functions for working with dates and times. It's used to timestamp attendance records.

These libraries, along with the face recognition model, enable the code to load images, detect faces, recognize known faces, mark attendance, and save the attendance records.

Yes, the Histogram of Oriented Gradients (HOG) model is indeed used in this code. Specifically, it's utilized in the face detection part.

Here's how it works:

1. **Face Detection**: The code employs the HOG model for detecting faces within the group photo. HOG is a feature descriptor that captures the local gradient information in an image, which is particularly effective for detecting objects like faces.

2. **Implementation**: The `face_recognition.face_locations()` function is used to find the locations of faces within the group photo. This function internally employs the HOG model to identify regions in the image that likely contain faces.

3. **Performance**: The HOG model is chosen here because it's relatively fast and efficient for face detection tasks. While it may not be as accurate as some deep learning-based face detectors, it strikes a good balance between speed and accuracy, making it suitable for real-time applications like attendance tracking.

In [ ]:
'''main code for smart attendance.'''

import os
from datetime import datetime
import cv2
import numpy as np
import pandas as pd
import face_recognition
from pathlib import Path

# Directory where the known faces are located
image_directory = Path("named_images")

# Get a list of all image files in the directory
image_files = [f for f in image_directory.iterdir() if f.suffix in ['.png', '.jpg', '.jpeg']]

# Initialize known_faces dictionary
known_faces = {}

# Load known faces
for image_file in image_files:
    name = image_file.stem  # Name of the person is the filename without the extension
    print(f"Loading image: {image_file}")
    face_encodings = face_recognition.face_encodings(face_recognition.load_image_file(str(image_file)))
    if face_encodings:
        known_faces[name] = face_encodings[0]
        print(f"Face encoding loaded for: {name}")
    else:
        print(f"No face encoding found for: {name}")

known_face_encodings = list(known_faces.values())
known_face_names = list(known_faces.keys())

# Initialize DataFrame to store attendance
attendance = pd.DataFrame(columns=['Name', 'Time', 'Status'])

# Add all known faces to the attendance DataFrame with status 'Absent'
for name in known_face_names:
    attendance = pd.concat([attendance, pd.DataFrame([{'Name': name, 'Time': None, 'Status': 'Absent'}])], ignore_index=True)

# Path to the group photo
group_photo_path = Path(r"C:\smart_learn-main\smart_attendance\gr878.jpg")  # replace 'grap.jpg' with your actual filename and extension

image = cv2.imread(str(group_photo_path))

if image is None:
    print(f"Could not open or find the image: {group_photo_path}")
else:
    print("Group photo loaded successfully.")
    small_frame = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Find the locations of all faces in the image
    face_locations = face_recognition.face_locations(rgb_small_frame, model='hog')

    print(f"Found {len(face_locations)} face(s) in the group photo.")

    # For each face location, extract the face encoding and compare it with the known face encodings
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = rgb_small_frame[top:bottom, left:right]
        face_encodings = face_recognition.face_encodings(face_image)
        if face_encodings:  # Check if a face encoding was returned
            face_encoding = face_encodings[0]

            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Mark the individual as present
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            attendance.loc[attendance['Name'] == name, ['Time', 'Status']] = [current_time, 'Present']
            print(f"Face recognized as {name}.")
        else:
            print("No face found in the group photo at this location")

# Save the attendance to an Excel file
now = datetime.now()
attendance_file_name = now.strftime("%Y-%m-%d_%H-%M-%S") + ".xlsx"
attendance.to_excel(attendance_file_name, index=False)
print(f"Attendance saved to {attendance_file_name}.")